In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/image-colorization/inception_resnet_v2_weights_tf_dim_ordering_tf_kernels.h5
/kaggle/input/image-colorization/ab/ab/ab3.npy
/kaggle/input/image-colorization/ab/ab/ab1.npy
/kaggle/input/image-colorization/ab/ab/ab2.npy
/kaggle/input/image-colorization/l/gray_scale.npy


In [2]:
import tensorflow as tf
from tensorflow.keras.layers import Conv2D,InputLayer, Conv2DTranspose, Dropout, BatchNormalization, Input, Concatenate, Activation, concatenate ,RepeatVector ,Reshape ,UpSampling2D
from keras.initializers import RandomNormal
from tensorflow.keras.models import Model, load_model, Sequential
from tensorflow.keras.optimizers import Adam
from skimage.transform import resize
from sklearn.model_selection import train_test_split
from tensorflow.keras.utils import plot_model
import numpy as np
from tensorflow.keras.callbacks import EarlyStopping,ModelCheckpoint
import cv2
import PIL
from skimage import transform
from PIL import Image
import random
import h5py
import os
import plotly.graph_objects as go
import matplotlib.pyplot as plt

In [ ]:
images_gray = np.load("../input/image-colorization/l/gray_scale.npy")
images_ab1 = np.load("../input/image-colorization/ab/ab/ab1.npy")
images_ab2 = np.load("../input/image-colorization/ab/ab/ab2.npy")
images_ab3 = np.load("../input/image-colorization/ab/ab/ab3.npy")


In [ ]:
#taking training data of 100
X_train = (images_gray[:100,:,:].astype('float')).reshape(100,224,224,1) #reshaping the input gray images
Y = (images_ab1[:100,:,:].astype('float'))


# Model

In [ ]:
weight_init = RandomNormal(stddev=0.02)
model = Sequential()
model.add(InputLayer(input_shape=(224, 224, 1)))
model.add(Conv2D(8, (3, 3), activation='relu', padding='same', strides=2,kernel_initializer=weight_init))
model.add(Conv2D(8, (3, 3), activation='relu', padding='same',kernel_initializer=weight_init))
model.add(Conv2D(16, (3, 3), activation='relu', padding='same',kernel_initializer=weight_init))
model.add(Conv2D(16, (3, 3), activation='relu', padding='same', strides=2))
model.add(Conv2D(32, (3, 3), activation='relu', padding='same',kernel_initializer=weight_init))
model.add(Conv2D(32, (3, 3), activation='relu', padding='same', strides=2,kernel_initializer=weight_init))
model.add(UpSampling2D((2, 2)))
model.add(Conv2D(32, (3, 3), activation='relu', padding='same',kernel_initializer=weight_init))
model.add(UpSampling2D((2, 2)))
model.add(Conv2D(16, (3, 3), activation='relu', padding='same',kernel_initializer=weight_init))
model.add(UpSampling2D((2, 2)))
model.add(Conv2D(2, (3, 3), activation='relu', padding='same',kernel_initializer=weight_init))
model.summary()



In [ ]:
# Finish model
model.compile(optimizer='rmsprop',loss='mse',metrics=['accuracy'])
#Train the neural network
history = model.fit(x=X_train, y=Y, batch_size=1, epochs=500)


In [ ]:
plt.plot(history.history['loss'])
plt.title('model loss')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.show()

In [ ]:
output = model.predict(X_train)

In [ ]:
LAB TO RGB

In [ ]:
def get_LAB(image_l, image_ab  ):
       
    image_l = image_l.reshape((224, 224, 1))
    image_lab = np.concatenate((image_l, image_ab), axis=2)
    image_lab = image_lab.astype("uint8")
 
    image_rgb = cv2.cvtColor(image_lab, cv2.COLOR_LAB2RGB)
    image_rgb = Image.fromarray(image_rgb)
    return image_rgb
def get_LAB1(image_l  ):
    image_ab =  np.ones((224,224,2))*128
    image_l = image_l.reshape((224, 224, 1))
    image_lab = np.concatenate((image_l, image_ab), axis=2)
    image_lab = image_lab.astype("uint8")
 
    image_rgb = cv2.cvtColor(image_lab, cv2.COLOR_LAB2RGB)
    image_rgb = Image.fromarray(image_rgb)
    return image_rgb

In [ ]:
Output

In [ ]:
for i in range(50):
    pred = get_LAB(X_train[i],output[i])
    real = get_LAB(X_train[i],Y[i])
    original = get_LAB1(X_train[i])
    f, axarr = plt.subplots(1,3)
    axarr[0].title.set_text('Black and white')  
    axarr[1].title.set_text('Prediction')  
    axarr[2].title.set_text('original')  
    axarr[0].imshow(original)
    axarr[1].imshow(pred)
    axarr[2].imshow(real)
    
    
    